In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [2]:
import pandas as pd
train = pd.read_csv("/Users/demilade.runsewe@aics.espritscholen.nl/Desktop/Fruit Wars Folder/Deep Learning/train_black_friday.csv", nrows=2000)
test = pd.read_csv("/Users/demilade.runsewe@aics.espritscholen.nl/Desktop/Fruit Wars Folder/Deep Learning/test_black_friday.csv", nrows=2000)
train

train["Product_ID"] = train["Product_ID"].astype("category").cat.codes
test["Product_ID"] = test["Product_ID"].astype("category").cat.codes


In [3]:
black_friday=pd.concat([train, test])
black_friday

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,219,F,0-17,10,A,2,0,3,NaN,NaN,8370.0
1,1000001,756,F,0-17,10,A,2,0,1,6.0,14.0,15200.0
2,1000001,261,F,0-17,10,A,2,0,12,NaN,NaN,1422.0
3,1000001,252,F,0-17,10,A,2,0,12,14.0,NaN,1057.0
4,1000002,866,M,55+,16,C,4+,0,8,NaN,NaN,7969.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1003308,756,F,18-25,20,B,3,1,5,11.0,12.0,NaN
1996,1003308,949,F,18-25,20,B,3,1,4,5.0,NaN,NaN
1997,1003308,45,F,18-25,20,B,3,1,5,NaN,NaN,NaN
1998,1003308,763,F,18-25,20,B,3,1,8,16.0,NaN,NaN


In [4]:
from sklearn.preprocessing import LabelEncoder
allcolumns = black_friday.select_dtypes(include = "object")
le = LabelEncoder()
for i in allcolumns.columns:
  black_friday[i] = le.fit_transform(black_friday[i])

black_friday

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,219,0,0,10,0,2,0,3,NaN,NaN,8370.0
1,1000001,756,0,0,10,0,2,0,1,6.0,14.0,15200.0
2,1000001,261,0,0,10,0,2,0,12,NaN,NaN,1422.0
3,1000001,252,0,0,10,0,2,0,12,14.0,NaN,1057.0
4,1000002,866,1,6,16,2,4,0,8,NaN,NaN,7969.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1003308,756,0,1,20,1,3,1,5,11.0,12.0,NaN
1996,1003308,949,0,1,20,1,3,1,4,5.0,NaN,NaN
1997,1003308,45,0,1,20,1,3,1,5,NaN,NaN,NaN
1998,1003308,763,0,1,20,1,3,1,8,16.0,NaN,NaN


In [5]:
black_friday["Product_Category_2"] = black_friday["Product_Category_2"].fillna(black_friday["Product_Category_2"].median())
black_friday["Product_Category_3"] = black_friday["Product_Category_3"].fillna(black_friday["Product_Category_3"].median())
black_friday["Purchase"] = black_friday["Purchase"].fillna(black_friday["Purchase"].median())
black_friday.info()
black_friday
features = black_friday.drop(columns = ["User_ID", "Product_ID", "Purchase"])
target = black_friday["Purchase"]

<class 'pandas.core.frame.DataFrame'>
Index: 4000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4000 non-null   int64  
 1   Product_ID                  4000 non-null   int16  
 2   Gender                      4000 non-null   int64  
 3   Age                         4000 non-null   int64  
 4   Occupation                  4000 non-null   int64  
 5   City_Category               4000 non-null   int64  
 6   Stay_In_Current_City_Years  4000 non-null   int64  
 7   Marital_Status              4000 non-null   int64  
 8   Product_Category_1          4000 non-null   int64  
 9   Product_Category_2          4000 non-null   float64
 10  Product_Category_3          4000 non-null   float64
 11  Purchase                    4000 non-null   float64
dtypes: float64(3), int16(1), int64(8)
memory usage: 382.8 KB


In [6]:
from sklearn.model_selection import train_test_split as tts
train_features, test_features, train_target, test_target = tts(features, target, train_size=0.7, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor()
rfc

RandomForestRegressor()

In [11]:
parameters = {"n_estimators":[50, 100, 200], "max_depth":[None, 10, 20, 30], "min_samples_split":[2, 5, 10]}
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(rfc, parameters, n_jobs=-1)
gs.fit(train_features, train_target)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [13]:
rfr = gs.best_estimator_
te = rfr.predict(test_features)

In [14]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(test_target, te)

2237.404238101858

In [15]:
model = Sequential ([
    Dense(200, activation="relu"),
    Dropout(0.3),
    Dense(100, activation="relu"),
    Dropout(0.3),
    Dense(50, activation="relu"), 
    Dense(1)
])

In [16]:
model.compile(optimizer=Adam(), loss="mean_absolute_error", metrics=["mean_absolute_error"])
model.fit(train_features, train_target, epochs=20)



Epoch 1/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8406.1670 - mean_absolute_error: 8406.1670
Epoch 2/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3113.1365 - mean_absolute_error: 3113.1365
Epoch 3/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2807.3933 - mean_absolute_error: 2807.3933
Epoch 4/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2669.2673 - mean_absolute_error: 2669.2673
Epoch 5/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2554.5181 - mean_absolute_error: 2554.5181
Epoch 6/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2611.7793 - mean_absolute_error: 2611.7793
Epoch 7/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2578.3569 - mean_absolute_error: 2578.3569
Epoch 8/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2467.4062 - mean_absolute_error: 2467.4062
Epoch 9/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2528.2141 - mean_absolute_error: 2528.2141
Epoch 10/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2483.5544 - mean_absolute_error:

In [17]:
mod = model.predict(test_features)
mean_absolute_error(test_target, mod)

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2329.064518432617